# LLM Zoomcamp 2024 Competition
This competition requires the use of LLMs to solve high school mathematics problems. The task is to develop models that can accurately solve the problems and then the predictions submitted.

In [1]:
# Import required modules.
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import os
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
import re
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
from scorer import score

/home/codespace/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set up OpenAI API key.
load_dotenv()

True

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
    temperature=0,
)

In [ ]:
message = completion.choices[0].message.content

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
    temperature=0,
)

In [3]:
# Application Design.
llm = OpenAI(model_name="gpt-4o", temperature=0, max_tokens=1500)

In [4]:
def get_answer(question):
    prompt = f"""Please solve the following high school math problem step by step. Explain your reasoning clearly and provide the final answer.

{question}

Step-by-step solution and final answer:"""

    response = llm.invoke(prompt)
    return response.strip()

In [5]:
# Function to Extract Answer.
def extract_numerical_answer(text):
    # Look for patterns like "Final answer: X" or "The answer is X" at the end of the text
    match = re.search(r'(?:final answer|the answer is)[:\s]*([+-]?\d*\.?\d+)', text, re.IGNORECASE)
    if match:
        return float(match.group(1))
    else:
        # If no clear final answer, look for the last number in the text
        numbers = re.findall(r'[+-]?\d*\.?\d+', text)
        return float(numbers[-1]) if numbers else None

In [6]:
# Function to Prepare Dataset and Get Answers.
pool = ThreadPoolExecutor(max_workers=6)

def map_progress(pool, seq, f):
    results = []

    with tqdm(total=len(seq)) as progress:
        futures = []

        for el in seq:
            future = pool.submit(f, el)
            future.add_done_callback(lambda p: progress.update())
            futures.append(future)

        for future in futures:
            result = future.result()
            results.append(result)

    return results

In [7]:
# create a function for processing each row.
def process_row(row):
    problem_id = row['problem_id']
    problem_text = row['problem_text']

    llm_reasoning = get_answer(problem_text)

    numerical_answer = extract_numerical_answer(llm_reasoning)

    return {
        'problem_id': problem_id,
        'problem_text': problem_text,
        'llm_reasoning': llm_reasoning,
        'answer': numerical_answer
    }

In [8]:
# Load the train data.
df_train = pd.read_csv('data/train.csv')
df_train.head()

,problem_id,problem_text,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,1.6
1,4723,"In a company of 30 people, 25 use the social n...",24
2,7135,The number of road traffic accidents (RTAs) in...,32
3,5814,Find the value of the expression $\dfrac{2\str...,256
4,9237,A traveler from Moscow wants to visit four cit...,53


In [9]:
# Process row.
rows = df_train.head().to_dict(orient='records')
process_row(rows[0])

NotFoundError: Error code: 404 - {'error': {'message': 'This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}

In [10]:
# Get df_results.
results = map_progress(pool, rows, process_row)
df_results = pd.DataFrame(results)
df_results

 40%|██████████████████████████████████████████                                                               | 2/5 [00:00<00:00,  6.70it/s]


NotFoundError: Error code: 404 - {'error': {'message': 'This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}

In [ ]:
# Show the head of train.
df_train.head()

In [ ]:
# Score the run.
score(df_train.head(), df_results, 'problem_id')

In [ ]:
# Put all together in a function.
def prepare_prompts_and_get_answers(df):
    rows = df.to_dict(orient='records')
    results = map_progress(pool, rows, process_row)
    return pd.DataFrame(results)

In [ ]:
# Return the results head.
df_train_results = prepare_prompts_and_get_answers(df_train)
df_train_results.head()

In [ ]:
# Get the total score.
score(df_train, df_train_results, 'problem_id')

In [ ]:
# Prepare the test submission.
df_test = pd.read_csv('data/test.csv')

df_test_results = prepare_prompts_and_get_answers(df_test)

submission = df_test_results[['problem_id', 'answer']]
submission.to_csv('starter_notebook_submission.csv', index=False)